In [1]:
import sys
sys.path.append("../")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from torch.nn.utils.rnn import pack_padded_sequence
import torchvision as tv
import torch.utils.data as td
from torch.nn import functional as F
from torch import nn
import torch

import models.nntools_modified as nt
import models.framework as fw
import models.framework_varying as fwv
import models.framework_single_varying as fwsv
import utils.config as cfg
from utils.datareader_toy import toyScenesdata

from utils.visualization import *

from PIL import Image
import cv2
# import imageio

cuda


In [2]:
net = fwv.MultiAgentNetwork(cfg.FCN_OUT, cfg.NUM_INTENTS, cfg.SCENE_OUT,
                                cfg.INTENT_IN, cfg.INTENT_OUT, cfg.SCORE_IN)

dataset = toyScenesdata(set_name="test", traj_time=cfg.PAST_TRAJECTORY_TIME)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = net.to(device)
test_set = toyScenesdata(set_name="test")
# output_dir = "data/res_gpu_original_downsample_epoch_75_pt_5/" # output_dir = cfg.EXP_NAME
output_dir = cfg.OUTPUT_PATH + cfg.EXP_NAME

# optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

Read in 1000 
700
200
100
Read in 1000 
700
200
100


In [3]:
def load_state_dict(checkpoint):
    """Loads the experiment from the input checkpoint."""
    net.load_state_dict(checkpoint['Net'])
    optimizer.load_state_dict(checkpoint['Optimizer'])
    history = checkpoint['History']

    # The following loops are used to fix a bug that was
    # discussed here: https://github.com/pytorch/pytorch/issues/2830
    # (it is supposed to be fixed in recent PyTorch version)
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.to(net.device)

def load():
    """Loads the experiment from the last checkpoint saved on disk."""
    checkpoint = torch.load(checkpoint_path,
                            map_location=net.device)
    load_state_dict(checkpoint)
#     del checkpoint

In [4]:
# test_loader = td.DataLoader(test_set, batch_size=cfg.BATCH_SIZE, shuffle=cfg.SHUFFLE, drop_last=cfg.DROP_LAST, pin_memory=True)

# Initialize history
history = []

# Define checkpoint paths
# if output_dir is None:
#     output_dir = 'experiment_{}'.format(time.time())
# os.makedirs(output_dir, exist_ok=True)
checkpoint_path = os.path.join(output_dir, "checkpoint.pth.tar")
config_path = os.path.join(output_dir, "config.txt")

# Transfer all local arguments/variables into attributes
# locs = {k: v for k, v in locals().items() if k is not 'self'}
# self.__dict__.update(locs)

# Load checkpoint and check compatibility
if os.path.isfile(config_path):
    with open(config_path, 'r') as f:
        if f.read()[:-1] != repr(self):
            raise ValueError(
                "Cannot create this experiment: "
                "I found a checkpoint conflicting with the current setting.")
    load()

In [ ]:
# TODO - replace these with the inputs used for the network
# ls - the entire groundtruth trajectory of the agent
# vs - the entire groundtruth velocities of the agent

# ls, vs -> required for plotting the actual trajectories
# vs additionally required for the initial velocity value
# s_points -> refers to the actual data that is used in init.npy file.
# these are the starting conditions set initially for the image
data_path = cfg.DATA_PATH + "2/2716/"
base_path = cfg.ACTUAL_PATH + "/2/2716/"
s_points = np.load(base_path + "init.npy", allow_pickle=True)
vs = np.load(base_path + "vs.npy")
ls = np.load(base_path + "ls.npy")
# im = imageio.imread("../data/toydataset/2/2716/scene.png")
im = np.array(cv2.imread(base_path + "scene.png"))
print(im.shape)

print("Displaying input to the network")
fig = plt.figure()
plt.imshow(im)
# plt.savefig("test_input.png")
plt.show()

# TODO - compute the new intents from the network
new_intents = np.array([2, 3])
f, f_new, c = visualize(new_intents, s_points, ls, vs)
# f.savefig("test_old.png")
# f_new.savefig("test_new.png")
print("Displaying Groundtruth complete trajectory")
f.show()
print("Displaying Predicted complete trajectory")
f_new.show()
if c:
    print("Collision occurred for new intents")
else:
    print("Successful new trajectory was generated")

In [5]:
# sample_img = imageio.imread("../data/toydataset_resampled/2/2716/scene.png")
sample_img = np.array(cv2.imread(data_path + "scene.png"))
sample_ls = np.load(data_path + "ls.npy")
sample_gt = np.array(list(np.array(np.load(data_path + "init.npy", allow_pickle=True))))
print(sample_img.shape, sample_ls.shape, sample_gt.shape)

(100, 100, 3) (2, 24) (2,)


In [6]:
sample_img = torch.from_numpy(sample_img)
sample_img = sample_img.view(1, *sample_img.size())
print(sample_img.shape)
sample_ls = torch.from_numpy(sample_ls)
sample_ls = sample_ls.view(1, *sample_ls.size())
print(sample_ls.shape)
sample_gt = torch.from_numpy(sample_gt)
sample_gt = sample_gt.view(1, *sample_gt.size())
print(sample_gt.shape)
sample_img = sample_img.to(net.device)
sample_ls = sample_ls.to(net.device)
sample_gt = sample_gt.to(net.device)

torch.Size([1, 100, 100, 3])
torch.Size([1, 2, 24])
torch.Size([1, 2])


In [19]:
y, d = net(sample_img, sample_ls, sample_gt)

Scene Output shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])
traj_output.shape:  torch.Size([1, 32])


In [20]:
print(y)
print(d)

tensor([[-0.1257, -0.1256, -0.1258, -0.1255, -0.1257, -0.1255, -0.1257, -0.1255,
         -0.1259, -0.1258, -0.1261, -0.1257, -0.1255, -0.1253, -0.1256, -0.1253]],
       device='cuda:0', grad_fn=<CopyBackwards>)
tensor(5, device='cuda:0')


In [18]:
_, predicted = torch.max(output[0], 1)
print(predicted)
# print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
#                               for j in range(4)))

tensor([15], device='cuda:0')


In [21]:
loss = net.criterion(y, d)
loss.backward()

IndexError: dimension specified as 0 but tensor has no dimensions

In [ ]:
s = nn.Softmax(dim=1)
sout = s(output)

In [ ]:
predicted_intents = list()
n_modes = output.shape[1]
n_agents = 0
nm = n_modes
while nm != 1:
    n_agents += 1
    nm = nm // 4
overall_ind = torch.argmax(sout, dim=1)
n_samples = overall_ind.shape[0]
for i in range(n_samples):
    ints = list()
    oin = oind[i]
    for k in range(1, n_agents+1):
        ints.append((oin%4**k).item())
        oin = oin//4**k
    predicted_intents.append(np.array(ints))
print(predicted_intents)
print(len(predicted_intents))

In [ ]:
for new_intents in predicted_intents:
    print(new_intents)
    f, f_new, c = visualize(new_intents, s_points, ls, vs)
    # f.savefig("test_old.png")
    # f_new.savefig("test_new.png")
    print("Displaying Groundtruth complete trajectory")
    f.show()
    print("Displaying Predicted complete trajectory")
    f_new.show()
    if c:
        print("Collision occurred for new intents")
    else:
        print("Successful new trajectory was generated")